# Functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def get_intensities():

  ###############################################
  # define intensities
  # intensity = np.sqrt([0,1,2,3])
  # intensity = np.array([0.7906, 1.1407, 1.4394, 0.0416])
  intensity = np.array([0.0088, 1.6027, 0.5473, 1.0638])
  ###############################################
  # Normalize ref constellation power to one
  pow_avg = np.sqrt(np.mean(np.abs(intensity)**2))  
  intensity_norm = intensity/pow_avg 

  return intensity_norm

def tx_PAM_Gen(syst): 
  '''
  This fuction generates PAM symbols based on the PAM order
  input: 
    syst: attribute dictionary of system parameters 
  output:    
    Tx_sym: the index of the symbols (any of [0,1,2,...,M]) where M is the PAM order
    Tx_coords: coordinates for Tx_sym
  '''   
  ###############################################
  # geterate random symbols with the size (N,1)
  Tx_syms = np.random.randint(0,syst.M,(syst.frame_size,1))

  ###############################################
  # get PAM reference coordinates
  ref_const = get_intensities()
  
  
  ###############################################
  # assign the intensities to the symbols 
  Tx_coords = ref_const[Tx_syms[:,0].astype(int)].reshape(-1,1)

  return Tx_syms, Tx_coords


def add_transmitter_noise(signal, syst, P_sig_before_PD, P_sig_after_PD):
  epsilon = 1e-40 
  noise = (np.random.randn(np.max(signal.shape))).reshape(-1,1)
  ASE_var=(1-syst.alpha)*(10**(-syst.SNR_total/10))*P_sig_after_PD
  var_n1 = ((-6*P_sig_before_PD)+np.sqrt((36*P_sig_before_PD**2)+12*ASE_var))/6
  print(var_n1)
  var_n1 =  np.max([epsilon, var_n1])
  
  # print(var_n1)
  Data_out_ASE = signal+np.sqrt(var_n1)*noise;
  return Data_out_ASE
  
def add_thermal_noise(signal, syst, P_sig_before_PD, P_sig_after_PD):
  noise = (np.random.randn(np.max(signal.shape))).reshape(-1,1)
  Thermal_var=(syst.alpha)*(10**(-syst.SNR_total/10))*P_sig_after_PD
  print(Thermal_var)
  Data_out=signal+np.sqrt(Thermal_var)*noise
  return Data_out
  

# Main

In [ ]:
def main(): 
  ###############################################
  # channel parameters 
  syst = AttrDict()    
  syst.M = 4
  syst.frame_size = int(1e6)
  
  for syst.alpha in np.linspace(0.0,0.1, num=1): # 10 valuses in [0,1] including 0 and 1 themselves.
    print('\n\n\t \u03B1= ', syst.alpha)
    print('\t---------------------------------')
    SER = []
    SNR = []
    for syst.SNR_total in range(14,23,1):
      Totla_S_error  = 0
      totoal_symbols = 0
      # count at least 100 symbol errors      
      while Totla_S_error<=100: 

        ###############################################
        # input symbols generation
        symbs, ref_PAM = tx_PAM_Gen(syst)


        P_sig_before_PD=np.mean(np.abs(ref_PAM)**2)
        P_sig_after_PD=np.mean(np.abs(ref_PAM)**4)
        ###############################################
        # add transmitter noise 
        data_Tx_noise = add_transmitter_noise(ref_PAM, syst, P_sig_before_PD, P_sig_after_PD)
        # data_Tx_noise = ref_PAM

        ###############################################
        # add jphoto detector effect 
        photo_detector = np.abs(data_Tx_noise)**2        

        ###############################################
        # add thermal noise 
        data_Thermal_noise = add_thermal_noise(photo_detector, syst, P_sig_before_PD, P_sig_after_PD).reshape(1,-1)
        # data_Thermal_noise = add_noise(photo_detector, syst).reshape(1,-1)
        ###############################################
        # calculate symbol errors
        # plt.figure(figsize=(8,2),dpi=200)    
        # plt.rcParams["font.family"] = "serif"     
        # plt.scatter(data_Thermal_noise, np.zeros_like(data_Thermal_noise),s=20, alpha = .5, label='Received symbols');       
        # plt.scatter(ref_PAM**2,np.zeros_like(ref_PAM)**2,s=5, alpha = .5, label ='Transmitted symbols');            
        # plt.legend()
        # plt.show()
        ref_const = get_intensities()

        # demodulate
        RX = []
        for i in range(np.max(data_Thermal_noise.shape)):
          idx = np.argmin(np.abs(data_Thermal_noise[0,i]-ref_const**2))       
          RX.append(idx) 
        RX = np.array(RX).reshape(-1,1)

        s_errors = np.sum(symbs!= RX)
        totoal_symbols+=np.max(symbs.shape)
        Totla_S_error += s_errors



          # plt.figure(dpi=200)
          # plt.scatter(data_Thermal_noise[0,:],np.zeros_like(data_Tx_noise)[:,0],s=2, alpha=0.5)
          # plt.scatter(ref_const**2,np.zeros_like(ref_const))
          # plt.show()
          
      SER.append(Totla_S_error/totoal_symbols)    
      SNR.append(syst.SNR_total)      
      print('\t SER for SNR= {} is {}'.format(SNR[-1], SER[-1]))

    SNR = np.array(SNR)
    SER = np.array(SER)
    print('\n\t \u03B1= ', syst.alpha)    
    print('SNR range is: \n',SNR)
    print('SER is: \n', SER)
if __name__ == "__main__":  
  main()



	 α=  0.0
	---------------------------------
0.01312760547279579
0.0
	 SER for SNR= 14 is 0.034013
0.010442197125387409
0.0
	 SER for SNR= 15 is 0.021953
0.008312460243602734
0.0
	 SER for SNR= 16 is 0.013238
0.006603229558966195
0.0
	 SER for SNR= 17 is 0.007326
0.0052456150012176
0.0
	 SER for SNR= 18 is 0.00393
0.004171185987738617
0.0
	 SER for SNR= 19 is 0.001789
0.003315010013574297
0.0
	 SER for SNR= 20 is 0.000731
0.0026333678318149665
0.0
	 SER for SNR= 21 is 0.00022
0.0020919763467954766
0.0
0.0020935325753635623
0.0
	 SER for SNR= 22 is 6.15e-05

	 α=  0.0
SNR range is: 
 [14 15 16 17 18 19 20 21 22]
SER is: 
 [3.4013e-02 2.1953e-02 1.3238e-02 7.3260e-03 3.9300e-03 1.7890e-03
 7.3100e-04 2.2000e-04 6.1500e-05]
